In [1]:
import csv
import numpy as np
import pandas as pd
import tensorflow as tf
from sentence_transformers import SentenceTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score

In [2]:
train_filepath = '/kaggle/input/clef2025-checkthat/data/english/train_en.tsv'
test_filepath = '/kaggle/input/clef2025-checkthat/data/english/dev_test_en.tsv'

In [3]:
train_data = pd.read_csv(train_filepath, sep='\t', quoting=csv.QUOTE_NONE)
test_data = pd.read_csv(test_filepath, sep='\t', quoting=csv.QUOTE_NONE)

In [4]:
vect = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.12k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
model = LogisticRegression(class_weight="balanced")
model.fit(X=vect.encode(train_data['sentence'].values), y=train_data['label'].values)

Batches:   0%|          | 0/26 [00:00<?, ?it/s]

LogisticRegression(class_weight='balanced')

In [6]:
predictions = model.predict(X=vect.encode(test_data['sentence'].values)).tolist()
pred_df = pd.DataFrame()
pred_df['sentence_id'] = test_data['sentence_id']
pred_df['label'] = predictions

Batches:   0%|          | 0/16 [00:00<?, ?it/s]

In [7]:
gold_values = test_data.label.values
pred_values = predictions

In [8]:
acc = accuracy_score(gold_values, pred_values)
m_prec, m_rec, m_f1, m_s = precision_recall_fscore_support(gold_values, pred_values, average="macro",
                                                           zero_division=0)
p_prec, p_rec, p_f1, p_s = precision_recall_fscore_support(gold_values, pred_values, labels=["SUBJ"],
                                                           zero_division=0)
roc_auc = roc_auc_score(gold_values, model.predict_proba(X=vect.encode(test_data['sentence'].values))[:, 1])

Batches:   0%|          | 0/16 [00:00<?, ?it/s]

In [9]:
acc, roc_auc

(0.7231404958677686, 0.7050312471696404)